# 특징 공학

실제로 머신러닝을 사용할 때 가장 중요한 단계 중 하나는 특징공학(feature engineering)이다. 특징 공학이란 문제에 대해 가지고 있는 정보를 모두 취해 특징 행렬을 구축하는데 사용할 수 있는 숫자로 변환하는 것이다.

## 범주 특징

비수치 데이터의 일반적인 유형 중 하나는 범주 데이터이다. 예를들어 '지역'과 같은 정보가 있다고 해보자. 해당 값에 서울, 뉴욕, 런던과 같은 정보가 있을때 이를 0, 1, 2 이런식으로 할수도 있지만 이는 좋은 선택이 아니다.

이 경우 이용할 수 있는 검증된 기법은 사실상 해당 범주의 유무를 각각 1이나 0으로 나타내는 추가 열을 생성하는 원-핫 인코딩(one-hot encoding)을 사용하는 것이다. 데이터가 딕셔너리의 리스트 형태라면 Scikit-learn의 `DictVectorizer`를 사용해 이 작업을 수행할 수 있다.

In [ ]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False, dtype=int)
vec.fit_transform(data)

지역 열이 세개의 레이블을 나타내는 세개의 분리된 열로 확장되었고 각 행에는 그 지역과 관련된 열에 1이라는 값이 들어가게 된다. 이렇게 scikit-learn 모델을 적합시키면 표준 방식으로 진행할 수 있다. 

각 열의 의미를 확인하려면 특징 이름을 검사하면 된다.

In [ ]:
vec.get_feature_names()

이 방식의 한가지 단점은 해당 범주에 들어가있는 값이 많은 경우 데이터 세트의 크기가 엄청나게 커질 수 있다는 것이다. 그러나 인코딩된 데이터가 대부분 0이기 때문에 희박한 결과가 매우 효율적인 해결책이 될 수 있다.

많은 사이킷런 추정기는 모델을 적합시키거나 평가할 때 이러한 희박한 입력값을 받는다. 사이킷런에는 이러한 유형의 인코딩을 지원하기 위해 `sklearn.preprocessing.OneHotEncoder`와 `sklearn.feature_extraction.FeatureHasher`가 포함되어 있다.

## 텍스트 특징

특징공학이 필요한 또 다른 경우는 텍스트를 대표 수치값의 집합으로 변환하는 것이다. 예를 들어 대부분의 자동화된 소셜 미디어 데이터 마이닝 기법은 텍스트를 숫자로 인코딩하는 형식에 의존한다. 데이터를 인코딩하는 가장 간단한 방법중 하나는 단어 세기(word count)를 이용하는 것이다. 즉, 각 텍스트 토막을 취해 그 안에 각 그 단어가 몇번 등장하는지 세고 그 결과를 테이블에 넣는 것이다. 

아래 예시를 살펴보자.

In [1]:
sample = ['problem of evil',
          'evil queen',
          'horizon problem']

단어 세기를 기반으로 이 데이터를 벡터화하려면 'problem', 'evil', 'horizon' 등의 단어를 나타내는 열을 만들어야 한다. 직접 할수도 있지만 사이킷런의 `CounterVectorizer`를 사용하면 지루한 작업을 피해갈 수 있다.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(sample)
X

<3x5 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

코드를 실행하면 각 단어가 등장하는 횟수를 기록하는 희박 행렬을 얻는다. 이를 레이블이 있는 열로 이뤄진 데이터 프레임으로 전환하면 검사하기가 더 쉬워진다.

In [4]:
import pandas as pd

pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

,evil,horizon,of,problem,queen
0,1,0,1,1,0
1,1,0,0,0,1
2,0,1,0,1,0


그러나 이 방식은 원시 단어 개수가 매우 자주 등장하는 단어에 너무 많은 가중치를 부여하는 특징을 가지게 한다는 문제가 있는데, 이것이 일부 분류 알고리즘에서는 최선의 방식이라고 볼 수 없다. 이 문제를 해결하는 한 가지 방법은 단어가 문서에서 얼마나 자주 등장하느냐에 따라 단어 수에 가중치를 부여하는 **TF-IDF**(term frequency-inverse document frequency, 단어 빈도-역문서 빈도)라는 기법이다. 이 특징을 계산하는 구문은 이전 예제와 비슷하다.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
X = vec.fit_transform(sample)
pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

,evil,horizon,of,problem,queen
0,0.517856,0.000000,0.680919,0.517856,0.000000
1,0.605349,0.000000,0.000000,0.000000,0.795961
2,0.000000,0.795961,0.000000,0.605349,0.000000
